In [1]:
from rubin_scheduler.scheduler import detailers
from lsst.ts.observing import ObservingBlock, ObservingScript
import numpy as np

In [2]:
name = "BLOCK-T248"
program = "BLOCK-T248"
reason = "comcam_science_imaging"
constraints = []
scripts = []

In [3]:
detailer = detailers.DitherDetailer(max_dither=0.2, per_night=False)

n_offsets = 30
night = 42
offsets = np.degrees(detailer._generate_offsets(n_offsets, night))

In [4]:
comcam_take_images_r_script = ObservingScript(
    name="maintel/take_image_comcam.py",
    standard=True,
    parameters= dict(
        filter="r_03",
        image_type="OBJECT",
        program=program,
        reason=reason,
        exp_times = 30.0,
        nimages = 1
    )
)

In [5]:
comcam_take_images_i_script = ObservingScript(
    name="maintel/take_image_comcam.py",
    standard=True,
    parameters= dict(
        filter="i_06",
        image_type="OBJECT",
        program=program,
        reason=reason,
        exp_times = 30.0,
        nimages = 1
    )
)

In [6]:
for offset in offsets:
    offset_script = ObservingScript(
        name="maintel/offset_mtcs.py",
        standard=True,
        parameters = dict(
            offset_xy = dict(
                x = offset[0]*3600.0,
                y = offset[1]*3600.0
            ),
            relative = False
        )
    )
    scripts.append(offset_script)
    scripts.append(comcam_take_images_r_script)
    scripts.append(comcam_take_images_i_script)

In [7]:
stop_tracking_script = ObservingScript(
    name="maintel/stop_tracking.py",
    standard=True,
    parameters = dict()
)
scripts.append(stop_tracking_script)

In [8]:
len(scripts)

91

In [9]:
block = ObservingBlock(
    name = name,
    program = program,
    scripts = scripts,
)